# 🎧 TOÀN BỘ CODE MẪU RECOMMENDER SYSTEM

## 1.Popularity-based Recommendation (cho user cold-start)
- Khi user mới, ta chỉ dựa vào độ phổ biến (số lượt rating + điểm trung bình).

In [1]:
import pandas as pd

# Thay vì tạo dữ liệu giả:
df = pd.read_csv('../data/data.csv')  # Đọc từ file
# Giả sử file có cột: user_id, song_id, rating
df.head(5)

,user_id,song_id,rating
0,1,18,5
1,3,12,4
2,3,9,3
3,3,15,1
4,3,12,2


In [2]:
import pandas as pd
import numpy as np

# ===== Giả lập dữ liệu =====
np.random.seed(0)
users = [f'U{i}' for i in range(1, 101)]
songs = [f'S{i}' for i in range(1, 21)]

data = []
for u in users:
    for s in np.random.choice(songs, size=np.random.randint(5, 15), replace=False):
        rating = np.random.randint(1, 6)
        data.append([u, s, rating])

df = pd.DataFrame(data, columns=['user_id', 'song_id', 'rating'])
print(df.head())

# ===== Tính điểm tổng hợp =====
summary = df.groupby('song_id').agg(
    avg_rating=('rating', 'mean'),
    count_rating=('rating', 'count')
).reset_index()

# Chuẩn hóa và tổng hợp
summary['score'] = 0.5 * (summary['avg_rating'] / 5) + 0.5 * (summary['count_rating'] / summary['count_rating'].max())

# Gợi ý top 5
top5 = summary.sort_values('score', ascending=False).head(5)
print("🎯 Top 5 bài hát gợi ý cho user cold-start:")
print(top5)


  user_id song_id  rating
0      U1     S16       2
1      U1     S11       2
2      U1     S15       1
3      U1     S14       2
4      U1      S2       5
🎯 Top 5 bài hát gợi ý cho user cold-start:
   song_id  avg_rating  count_rating     score
10     S19    3.163636            55  0.816364
12     S20    3.408163            49  0.786271
19      S9    2.925926            54  0.783502
11      S2    3.224490            49  0.767904
13      S3    3.000000            51  0.763636


## 2.Item–Item Collaborative Filtering (user đã nghe 1 bài)
- Dựa vào đồng xuất hiện (co-occurrence) của các bài hát cùng được nghe bởi nhiều người.

In [3]:
# Giả sử user đã nghe bài S1
target_song = 'S1'

# Tạo ma trận người dùng - bài hát (1 nếu có nghe)
matrix = pd.crosstab(df['user_id'], df['song_id'])
co_occurrence = matrix.T.dot(matrix)  # nhân ma trận để tính số user cùng nghe
np.fill_diagonal(co_occurrence.values, 0)  # bỏ chính nó

# Tính top 5 bài có đồng xuất hiện cao nhất với S1
top_related = co_occurrence[target_song].sort_values(ascending=False).head(5)
print("🎧 Top 5 bài hát liên quan đến", target_song)
print(top_related)


🎧 Top 5 bài hát liên quan đến S1
song_id
S4     25
S16    24
S7     24
S11    23
S13    23
Name: S1, dtype: int64


## 3.User–User Collaborative Filtering (user warm-start)
- Gợi ý dựa vào user tương tự bằng Cosine similarity giữa vector rating.

In [4]:
from sklearn.metrics.pairwise import cosine_similarity

# Ma trận user–song (điền 0 nếu chưa nghe)
rating_matrix = df.pivot_table(index='user_id', columns='song_id', values='rating', fill_value=0)

# Tính độ tương đồng giữa user
similarity = pd.DataFrame(cosine_similarity(rating_matrix), 
                          index=rating_matrix.index, 
                          columns=rating_matrix.index)

target_user = 'U1'
similar_users = similarity[target_user].sort_values(ascending=False).iloc[1:6]
print("👯 User tương tự với", target_user)
print(similar_users)

# Gợi ý bài hát user chưa nghe nhưng user tương tự đã nghe
user_rated = rating_matrix.loc[target_user]
recommend = rating_matrix.loc[similar_users.index].mean().sort_values(ascending=False)
recommend = recommend[user_rated == 0].head(5)
print("🎵 Gợi ý top 5 cho", target_user)
print(recommend)


👯 User tương tự với U1
user_id
U29    0.750000
U72    0.737756
U93    0.690166
U73    0.667298
U40    0.653305
Name: U1, dtype: float64
🎵 Gợi ý top 5 cho U1
song_id
S17    2.0
S10    1.4
S19    1.2
S3     1.0
S5     0.8
dtype: float64


## 4.Content-based Recommendation (dựa vào mô tả bài hát)
- Gợi ý bài tương tự bằng TF-IDF + Cosine Similarity trên nội dung.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Giả lập dữ liệu mô tả bài hát
songs_info = pd.DataFrame({
    'song_id': songs,
    'description': [
        "pop upbeat dance", "sad ballad", "rock guitar", "jazz smooth",
        "hiphop rap", "pop love song", "classical piano", "electronic edm",
        "acoustic", "indie chill", "r&b soul", "country guitar",
        "folk acoustic", "metal rock", "instrumental", "electro pop",
        "vietnamese ballad", "remix edm", "lofi chill", "pop female vocal"
    ]
})

# TF-IDF vectorization
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(songs_info['description'])
similarity_matrix = cosine_similarity(tfidf_matrix)

# Tạo DataFrame similarity
sim_df = pd.DataFrame(similarity_matrix, index=songs, columns=songs)

# Gợi ý bài tương tự với bài S1
target = 'S1'
top_similar = sim_df[target].sort_values(ascending=False).iloc[1:6]
print("🎧 Top 5 bài hát tương tự theo nội dung với", target)
print(top_similar)


🎧 Top 5 bài hát tương tự theo nội dung với S1
S16    0.268625
S6     0.208841
S20    0.208841
S4     0.000000
S3     0.000000
Name: S1, dtype: float64


## 5.Hybrid Recommendation (phối hợp popularity + content)
- Kết hợp giữa độ phổ biến và độ tương đồng nội dung.

In [6]:
# Giả sử user cold-start nhưng thích bài "S1"
popularity = summary.set_index('song_id')['score']
content_sim = sim_df['S1']

# Chuẩn hóa và kết hợp
combined = 0.6 * content_sim + 0.4 * popularity.reindex(content_sim.index, fill_value=0)
hybrid_top5 = combined.sort_values(ascending=False).iloc[1:6]

print("⚗️ Top 5 bài hát hybrid gợi ý cho user thích S1:")
print(hybrid_top5)


⚗️ Top 5 bài hát hybrid gợi ý cho user thích S1:
S20    0.439813
S16    0.436274
S6     0.390200
S19    0.326545
S9     0.313401
dtype: float64


## 6.Matrix Factorization (SVD – mô hình gợi ý nâng cao)
- Học các nhân tử ẩn từ ma trận rating.

In [7]:
from sklearn.decomposition import TruncatedSVD

R = rating_matrix.values
svd = TruncatedSVD(n_components=10, random_state=42)
latent = svd.fit_transform(R)
pred = np.dot(latent, svd.components_)

pred_df = pd.DataFrame(pred, index=rating_matrix.index, columns=rating_matrix.columns)
print("🎯 Gợi ý top 5 cho U1 bằng SVD:")
print(pred_df.loc['U1'].sort_values(ascending=False).head(5))


🎯 Gợi ý top 5 cho U1 bằng SVD:
song_id
S2     4.247911
S13    4.015733
S20    3.738225
S7     2.550986
S16    2.488953
Name: U1, dtype: float64


## 7.Cold-start Item Handling (item mới chưa có rating)
- Khi bài hát mới thêm vào, chưa ai nghe → không có dữ liệu CF.
- Giải pháp: dùng nội dung (content-based) hoặc đề xuất tạm dựa vào popularity.

In [ ]:
new_song = {'song_id': 'S21', 'description': 'pop dance upbeat remix'}
songs_info = pd.concat([songs_info, pd.DataFrame([new_song])], ignore_index=True)

# Vector hóa lại để tính similarity với S21
tfidf_matrix = tfidf.fit_transform(songs_info['description'])
similarity_matrix = cosine_similarity(tfidf_matrix)
sim_df = pd.DataFrame(similarity_matrix, index=songs_info['song_id'], columns=songs_info['song_id'])

print("🆕 Item mới S21 tương tự nhất với:")
print(sim_df['S21'].sort_values(ascending=False).iloc[1:6])


🆕 Item mới S21 tương tự nhất với:
song_id
S1     0.849399
S18    0.373177
S16    0.227257
S6     0.175024
S20    0.175024
Name: S21, dtype: float64


: 

## Lý thuyết

| **Nhóm chủ đề**           | **Câu hỏi thường gặp**                                    | **Mục tiêu đánh giá / Gợi ý trả lời**                                                    |
| ------------------------- | --------------------------------------------------------- | ---------------------------------------------------------------------------------------- |
| 🧊 **Cold-start problem** | 1. Cold-start là gì? Có mấy loại cold-start?              | Có 3 loại: user cold-start, item cold-start, system cold-start.                          |
|                           | 2. Vì sao cold-start là vấn đề trong recommender systems? | Vì hệ gợi ý cần dữ liệu lịch sử, khi không có dữ liệu thì không thể suy luận tương đồng. |
|                           | 3. Cách khắc phục cold-start user?                        | Dựa vào popular items, demographic info, hỏi người dùng chọn sở thích ban đầu.           |
|                           | 4. Cách khắc phục cold-start item?                        | Dựa vào metadata nội dung (thể loại, mô tả), content-based, hybrid approach.             |
|                           | 5. Nếu thêm item mới vào hệ thống thì xử lý thế nào?      | Dùng content-based tạm thời hoặc chiến lược “exploration” để thu rating ban đầu.         |
|                           | 6. Nếu thêm user mới chưa có lịch sử thì xử lý sao?       | Dùng global average, questionnaire, demographic-based, hoặc hybrid tạm thời.             |
|                           | 7. Cold-start và sparsity khác nhau thế nào?              | Cold-start: thiếu dữ liệu *ban đầu*; Sparsity: dữ liệu *thưa thớt* trên toàn hệ thống.   |
|                           | 8. Giải pháp “active learning” trong cold-start là gì?    | Hệ thống chủ động hỏi người dùng mới để có dữ liệu khởi tạo.                             |


| **Nhóm chủ đề**                     | **Câu hỏi thường gặp**                             | **Gợi ý trả lời**                                                                   |
| ----------------------------------- | -------------------------------------------------- | ----------------------------------------------------------------------------------- |
| 👥 **Collaborative Filtering (CF)** | 1. Collaborative Filtering là gì?                  | Dự đoán sở thích dựa trên hành vi của người dùng tương tự.                          |
|                                     | 2. Phân biệt User-based CF và Item-based CF?       | User-based dựa vào người dùng tương tự; Item-based dựa vào sản phẩm tương tự.       |
|                                     | 3. Ưu nhược điểm của CF?                           | Ưu: không cần metadata. Nhược: cold-start, sparsity, scalability.                   |
|                                     | 4. Công thức tính độ tương đồng trong CF?          | Cosine, Pearson, Jaccard.                                                           |
|                                     | 5. Ma trận user–item dùng để làm gì?               | Dùng làm cơ sở tính tương đồng và dự đoán rating.                                   |
|                                     | 6. Co-occurrence matrix khác gì similarity matrix? | Co-occurrence: đếm số lần đồng xuất hiện; similarity: chuẩn hóa theo độ tương đồng. |
|                                     | 7. Khi nào dùng item–item CF thay vì user–user CF? | Khi số lượng item < user hoặc dữ liệu user thay đổi nhiều.                          |
|                                     | 8. CF dựa trên implicit feedback là gì?            | Dựa vào click/view thay vì rating (ALS, BPR).                                       |


| **Nhóm chủ đề**                      | **Câu hỏi thường gặp**                          | **Gợi ý trả lời**                                                  |
| ------------------------------------ | ----------------------------------------------- | ------------------------------------------------------------------ |
| 📚 **Content-based Filtering (CBF)** | 1. Content-based là gì?                         | Gợi ý dựa vào đặc trưng nội dung của item mà user đã thích.        |
|                                      | 2. Ưu điểm của CBF so với CF?                   | Không bị cold-start user, có thể gợi ý item mới có mô tả.          |
|                                      | 3. Nhược điểm của CBF?                          | Thiếu đa dạng, khó mở rộng nếu dữ liệu content ít hoặc trùng lặp.  |
|                                      | 4. Các kỹ thuật biểu diễn nội dung thường dùng? | TF-IDF, Word2Vec, embedding, metadata vector.                      |
|                                      | 5. Khi nào nên kết hợp CF và CBF?               | Khi muốn khắc phục cold-start và tận dụng cả nội dung lẫn hành vi. |


| **Nhóm chủ đề**              | **Câu hỏi thường gặp**               | **Gợi ý trả lời**                                        |
| ---------------------------- | ------------------------------------ | -------------------------------------------------------- |
| ⚗️ **Hybrid Recommendation** | 1. Hybrid recommender là gì?         | Kết hợp nhiều kỹ thuật (CF + CBF + Popularity).          |
|                              | 2. Các cách kết hợp hybrid phổ biến? | Weighted hybrid, Switching hybrid, Cascade hybrid.       |
|                              | 3. Ưu điểm của hybrid?               | Giảm cold-start, tăng chính xác, đa dạng hơn.            |
|                              | 4. Ví dụ tình huống hybrid?          | Dùng content-based cho item mới, CF cho user có lịch sử. |


| **Nhóm chủ đề**                   | **Câu hỏi thường gặp**         | **Gợi ý trả lời**                                                       |
| --------------------------------- | ------------------------------ | ----------------------------------------------------------------------- |
| 🧮 **Matrix Factorization / SVD** | 1. Matrix factorization là gì? | Phân rã ma trận rating thành 2 vector ẩn (user & item latent features). |
|                                   | 2. Ưu điểm của SVD?            | Giảm chiều, khắc phục sparsity, tăng độ chính xác.                      |
|                                   | 3. Nhược điểm?                 | Cần dữ liệu đủ lớn, không xử lý tốt cold-start.                         |
|                                   | 4. Công thức SVD cơ bản?       | R ≈ UΣVᵀ (R là ma trận rating).                                         |


| **Nhóm chủ đề**             | **Câu hỏi thường gặp**                                | **Gợi ý trả lời**                                                  |
| --------------------------- | ----------------------------------------------------- | ------------------------------------------------------------------ |
| 🧠 **Evaluation & Metrics** | 1. Các chỉ số đánh giá recommender thường dùng?       | RMSE, MAE, Precision@K, Recall@K, F1, MAP, NDCG.                   |
|                             | 2. Khác biệt giữa RMSE và Precision@K?                | RMSE đo lỗi rating dự đoán; Precision@K đo chất lượng top N gợi ý. |
|                             | 3. Tại sao cần chia dữ liệu train/test?               | Để mô hình không bị overfit và đánh giá khách quan.                |
|                             | 4. Cross-validation trong recommender dùng để làm gì? | Kiểm tra độ ổn định của mô hình với dữ liệu khác nhau.             |


| **Nhóm chủ đề**           | **Câu hỏi thường gặp**                                    | **Gợi ý trả lời**                                                          |
| ------------------------- | --------------------------------------------------------- | -------------------------------------------------------------------------- |
| 🚀 **Ứng dụng & Mở rộng** | 1. Gợi ý được ứng dụng ở đâu?                             | Netflix, Spotify, Shopee, YouTube, TikTok, v.v.                            |
|                           | 2. Recommender online vs offline khác nhau thế nào?       | Online cập nhật liên tục theo hành vi, offline huấn luyện định kỳ.         |
|                           | 3. Làm sao đảm bảo gợi ý đa dạng?                         | Sử dụng diversification metrics hoặc penalize trùng lặp.                   |
|                           | 4. Làm sao cập nhật mô hình khi có dữ liệu mới?           | Incremental learning, retrain định kỳ, hoặc dùng online learning.          |
|                           | 5. Tại sao nói recommender system liên quan đến Big Data? | Vì dữ liệu người dùng – sản phẩm rất lớn, cần hệ phân tán (Spark, Hadoop). |
